In [ ]:
!pip install --quiet cornac


     |████████████████████████████████| 12.4 MB 15.8 MB/s 


In [ ]:
import sys 
import cornac
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from cornac.eval_methods import RatioSplit, BaseMethod, CrossValidation,StratifiedSplit
from cornac.models import MF, NMF, WMF, BPR, CTR
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch
from cornac.data import Reader

%tensorflow_version 1.x
import tensorflow as tf

SEED = 42
VERBOSE = True

TensorFlow 1.x selected.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
review_train=pd.read_csv('/content/gdrive/MyDrive/train_review.csv')
review_train.head()

,overall,reviewTime,reviewerID,asin,style,reviewText,summary,unixReviewTime
0,5.0,"03 22, 2016",A2MOB79WI93FDN,6040972467,"{'Size:': ' X-Large', 'Color:': ' Pink'}",love it,Five Stars,1458604800
1,3.0,"11 19, 2016",A16O44MQ3ZS7CL,6040972467,"{'Size:': ' 4/6', 'Color:': ' Pink'}",This was too big for me in the bust area (and ...,Eh. Lol. Too big for my little size.,1479513600
2,3.0,"09 17, 2016",A14ZLDIG0XD6AL,6040972467,"{'Size:': ' 12/14', 'Color:': ' White and Blue'}","This dress fit fairly well, except that I'm pr...",Nice dress - fairly deep V,1474070400
3,3.0,"08 21, 2016",A339XOY8DU2UWE,6040972467,"{'Size:': ' 12/14', 'Color:': ' White And Red'}","The fit on this is awesome, and I love the des...","The fit on this is awesome, and I love the design",1471737600
4,5.0,"08 19, 2016",A2BLA3KO8HGANN,6040972467,"{'Size:': ' 16', 'Color:': ' White And Red'}",I am very pleased with this dress! I wasn't su...,Perfect Comfortable Summer Dress!,1471564800


In [ ]:
review_test=pd.read_csv('/content/gdrive/MyDrive/test_seen_review.csv')
review_test.head()

,overall,reviewTime,reviewerID,asin,style,reviewText,summary,unixReviewTime
0,3.0,"09 5, 2017",A0003492LQH8LJXPWDMZ,B01FY9XN00,"{'Size:': ' 3', 'Color:': ' Tan'}","Decent material, the shorts were a bit bigger ...",Overall not bad for the price,1504569600
1,2.0,"02 16, 2017",A001170867ZBE9FORRQL,B00G2CNYNA,"{'Size:': ' Large', 'Color:': ' Water Blue'}","Nice material, but fits very small - unable to...",Two Stars,1487203200
2,5.0,"02 22, 2016",A0036810AKGSUKHOLV23,B00UBOPM6W,"{'Size:': ' Medium', 'Color:': ' Black White'}",Great dress fit great,Five Stars,1456099200
3,5.0,"01 18, 2017",A0036810AKGSUKHOLV23,B017XB96G2,"{'Size:': ' Small', 'Color:': ' #06 Black'}",Great sweater!!!,Five Stars,1484697600
4,4.0,"10 26, 2016",A0039616ADOZ0KMWQRNX,B01B5DLI88,"{'Size:': ' X-Large', 'Color:': ' Red/Navy'}",I thought these would be a thicker material fo...,Long and thin but still a win!,1477440000


In [ ]:
print(review_train.shape)
print(review_test.shape)

(1069882, 8)
(191831, 8)


In [ ]:
# Select only user_id, item_id and rating columns from reviews dataset

rating_triplet_train = review_train[['reviewerID', 'asin', 'overall']]
rating_triplet_train.columns = ['user_id', 'item_id', 'rating']


rating_triplet_test = review_test[['reviewerID', 'asin', 'overall']]
rating_triplet_test.columns = ['user_id', 'item_id', 'rating']

In [ ]:
# Convert review data to Cornac format 

train = list(rating_triplet_train.to_records(index=False))
test = list(rating_triplet_test.to_records(index=False))

In [ ]:
base_method = BaseMethod.from_splits(
    train_data=train, test_data=test, exclude_unknowns=True, verbose=VERBOSE,rating_threshold=1.0, seed=SEED
)



rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 16713 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 505047
Number of items = 42362
Number of ratings = 1053169
Max rating = 5.0
Min rating = 1.0
Global mean = 4.1
---
Test data:
Number of users = 168775
Number of items = 27914
Number of ratings = 191113
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 505047
Total items = 42362


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 126 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [ ]:
eval_metrics = [
  cornac.metrics.Recall(k=20),
  cornac.metrics.NCRR(k=20),
  cornac.metrics.NDCG(k=20),
]

In [ ]:
# BPR model
K=1000
bpr = BPR(k=1000, max_iter=500, learning_rate=0.1, lambda_reg=0.05, 
          verbose=VERBOSE, seed=SEED, name=f"BPR(K={K})")

# rs = RatioSplit(reviews_dataset, test_size=0.2, seed=SEED, verbose=VERBOSE)
# cornac.Experiment(eval_method=rs, models=[bpr], metrics=eval_metrics).run()

In [ ]:
cornac.Experiment(
    eval_method=base_method,
    models=[bpr],
    metrics=eval_metrics
).run()


[BPR(K=1000)] Training started!



Optimization finished!

[BPR(K=1000)] Evaluation started!




TEST:
...
            | NCRR@20 | NDCG@20 | Recall@20 | Train (s) |  Test (s)
----------- + ------- + ------- + --------- + --------- + ---------
BPR(K=1000) |  0.1043 |  0.1374 |    0.2465 | 1234.9510 | 2277.6290



A2522ZTCKCJZQT
A2PACJHFKI74Y3
A2OSLOUKH5VWGL
A1HV6KKVTGMCGE

In [ ]:
# get top 10 recommended items asin for selection users user_id
l=["A2522ZTCKCJZQT", "A2PACJHFKI74Y3", "A2OSLOUKH5VWGL", "A1HV6KKVTGMCGE"]
ind_l = [rating_triplet_test[rating_triplet_test['user_id']==i].index.values.astype(int)[0] for i in l]
print(ind_l)
# [57908, 86544, 85868, 25228]
item_idx=[]
for i in ind_l:
  # print(len(bpr.rank(i)[0][:10].tolist()))
  item_idx.append(bpr.rank(i)[0][:10].tolist())
print(item_idx)

asin_l=[]
for i in item_idx:
  asins=[]
  for j in i:
    asins.append(rating_triplet_test.iloc[j,1])
  asin_l.append(asins)
print(asin_l)


[57908, 86544, 85868, 25228]
[[11671, 20260, 18575, 24179, 11606, 16770, 17130, 5503, 12638, 23424], [8369, 317, 2193, 12697, 3295, 1869, 1224, 1204, 863, 12827], [1798, 317, 852, 15071, 12827, 1204, 5237, 843, 863, 2193], [375, 888, 486, 377, 12827, 3154, 372, 374, 317, 863]]
[['B00WIS9CEC', 'B00T8R0F4C', 'B00XC2FB66', 'B01E5Q8IE4', 'B006WMAC2Q', 'B0001MJG02', 'B00ZN0A00M', 'B00016QOX0', 'B01HIRG2OA', 'B01CYAUZ94'], ['B015FFTIVQ', 'B00453ER48', 'B01AVQF0DO', 'B00WW6YJG6', 'B008VJDUEA', 'B0018OHL5E', 'B010FC88DS', 'B00IRW79UM', 'B00NQJ5LAQ', 'B0008EOC5W'], ['B0012F7FGE', 'B00453ER48', 'B0009XDAH8', 'B00K58JCVU', 'B0008EOC5W', 'B00IRW79UM', 'B01EA9OZDO', 'B015IO2VYK', 'B00NQJ5LAQ', 'B01AVQF0DO'], ['B00OC0K6EE', 'B01E61CD1C', 'B00JMY6UW2', 'B00W4E3ZZC', 'B0008EOC5W', 'B00DFMH9EK', 'B003OQUKHK', 'B013LASMT6', 'B00453ER48', 'B00NQJ5LAQ']]


In [ ]:
# MF model

K = 50
wmf = WMF(k=K, max_iter=20, learning_rate=0.1,
        verbose=VERBOSE, seed=SEED, name=f"WMF(K={K})")

cornac.Experiment(eval_method=base_method, models=[wmf], metrics=eval_metrics).run()


[WMF(K=50)] Training started!



Learning completed!

[WMF(K=50)] Evaluation started!




TEST:
...
          | NCRR@20 | NDCG@20 | Recall@20 | Train (s) |  Test (s)
--------- + ------- + ------- + --------- + --------- + ---------
WMF(K=50) |  0.0011 |  0.0014 |    0.0025 | 6882.2817 | 1437.2023

